# 互评作业3：基于支持向量机的手写数字识别

项目地址：https://github.com/Fourseas54/DM3

In [ ]:
!pip install scikit-learn

In [1]:
import numpy as np 
import struct 
from PIL import Image 
import os

## 读取 MNIST 数据

MNIST数据集并非由一般的图片格式组成，需要转化为图片，这里将其转换后保存为.png格式

In [2]:
def make_img(fn_img = "train-images.idx3-ubyte", 
             fn_label = "train-labels.idx1-ubyte", 
             size1 = 47040016, size2 = 60008, root = "train"):
    fmt_img1 = ">IIII"
    offset_img1 = offset_label1= 0
    fmt_label1 = ">II"
    data_file_size = str(size1 - 16) + "B"
    labels_file_size = str(size2 - 8) + "B"
    fmt_img2 = ">" + data_file_size
    offset_img2 = struct.calcsize(fmt_img1)
    fmt_label2 = ">" + labels_file_size
    offset_label2 = struct.calcsize(fmt_label1)
    with open(fn_img, 'rb') as f:
        data_buf = f.read()
    with open(fn_label, 'rb') as f:
        label_buf = f.read()
    magic_img, numImages, numRows, numColumns = struct.unpack_from(fmt_img1, data_buf, offset_img1) 
    datas = struct.unpack_from(fmt_img2, data_buf, offset_img2)
    datas = np.array(datas).astype(np.uint8).reshape(numImages, 1, numRows, numColumns)
    magic_label, numLabels = struct.unpack_from(fmt_label1, label_buf, offset_label1)
    labels = struct.unpack_from(fmt_label2, label_buf, offset_label2)
    labels = np.array(labels).astype(np.int64)
    if not os.path.exists(root): 
        os.mkdir(root) 
    for i in range(10): 
        file_name = root + os.sep + str(i) 
        if not os.path.exists(file_name): 
            os.mkdir(file_name) 
    for ii in range(numLabels): 
        img = Image.fromarray(datas[ii, 0, 0:28, 0:28]) 
        label = labels[ii] 
        file_name = root + os.sep + str(label) + os.sep + str(ii).zfill(5) + '.png' 
        img.save(file_name)

## 划分训练集与测试集

本身数据集就已经分为训练集和测试机，比例约为6:1
分别加载并对其数据格式进行转换

In [3]:
make_img("train-images.idx3-ubyte","train-labels.idx1-ubyte",47040016,60008,"train")
make_img("t10k-images.idx3-ubyte","t10k-labels.idx1-ubyte",7840016,10008,"test")

In [14]:
from sklearn.svm import SVC
import joblib
from joblib import dump,load
from sklearn.metrics import confusion_matrix, classification_report
import glob
import time

##  图片向量化与特征缩放

DataLoader类用于将.png格式图片转换为向量，
并对其进行归一化，转换为0-1之间的值

In [7]:
class DataLoader(object):
    def get_data_labels(self, fpath = "train"):
            paths = glob.glob(fpath + os.sep + "*")
            X = []
            y = []
            for fpath in paths:
                fs = self.get_files(fpath)
                
                for fn in fs:
                    X.append(self.img2vec(fn))
                label = np.repeat(int(os.path.basename(fpath)), len(fs))
                y.append(label)
            labels = y[0]
            for i in range(len(y) - 1):
                labels = np.append(labels, y[i + 1])
            return np.array(X), labels
    def img2vec(self, fn):
            #将jpg等格式的图片转为向量
            im = Image.open(fn).convert('L')
            #归一化
            im = Image.eval(im,(lambda x: x/float(254)))
            im = im.resize((28,28))
            tmp = np.array(im)
            vec = tmp.ravel()
            return vec
    def get_files(self,dirs):
            fpath = []
            for file in os.listdir(dirs):
                file = os.path.join(dirs,file)
                fpath.append(file)
            return fpath

 ## 构建支持向量机模型

### 构建训练器模型

In [8]:
class Trainer(object):
    '''训练器;'''
    def svc(self, x_train, y_train):
        '''构建分类器'''
        model = SVC(kernel = 'poly',degree = 4,probability= True)
        model.fit(x_train, y_train)
        return model
    def save_model(self,model,filename):
        dump(model, filename) 
    def load_model(self,filename):
        clf = load(filename)
        return clf

### 构建测试模型

In [9]:
class Tester(object):
    '''测试器;'''
    def __init__(self, model_path):
        trainer = Trainer()      
        self.clf = trainer.load_model(model_path)
    def clf_metrics(self,X_test,y_test):
        """评估分类器效果"""
        pred = self.clf.predict(X_test)
        cnf_matrix = confusion_matrix(y_test, pred)
        score = self.clf.score(X_test, y_test)
        clf_repo = classification_report(y_test, pred)
        return cnf_matrix, score, clf_repo

### 进行训练

In [10]:
loader = DataLoader()
trainer = Trainer()
X, y = loader.get_data_labels()
clf = trainer.svc(X, y)

### 保存模型，并进行测试

In [15]:
trainer.save_model(clf, "mnist_svm.m")
X_test, y_test = loader.get_data_labels("test")
tester = Tester("mnist_svm.m")

## 模型评估

In [ ]:
mt, score, repo = tester.clf_metrics(X_test, y_test)


In [21]:
print("\n混淆矩阵:\n\n",mt)
print("\nR2评估指数:\n\n",score)
print("\nprecision,recall以及F1-score:\n\n",repo)


混淆矩阵:

 [[ 960    0    1    0    0   11    6    1    1    0]
 [   0 1123    2    2    0    1    4    0    3    0]
 [   6   19  978    2    1    1    5   12    8    0]
 [   0   10    1  974    0    9    0    9    4    3]
 [   1   11    3    0  949    0    5    1    1   11]
 [   4    5    3   10    1  857    5    0    6    1]
 [   5   16    1    0    5    6  923    0    2    0]
 [   0   32    9    2    2    0    0  973    0   10]
 [   3    6    1   10    7   13    0    2  928    4]
 [   5   12    1    4   18    4    1    7    5  952]]

R2评估指数:

 0.9617

precision,recall以及F1-score:

               precision    recall  f1-score   support

           0       0.98      0.98      0.98       980
           1       0.91      0.99      0.95      1135
           2       0.98      0.95      0.96      1032
           3       0.97      0.96      0.97      1010
           4       0.97      0.97      0.97       982
           5       0.95      0.96      0.96       892
           6       0.97      0.9